# ETL

En esta jupyter notebook se desarrolla la extracción, transformación y carga de los conjuntos de datos.

## Importaciones

In [1]:
import pandas as pd
import json
import gzip
import numpy as np


%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

## Extracción de los datos

Funcion para extraer los datos ".json" y convertirlos en ".csv" para la exploración.

In [8]:
def procesar_archivo_json(file_path, file_path1):
    data_list = []

    try:
        with gzip.open(file_path, 'rb') as file:
            for line in file:
                try:
                    decoded_line = line.decode('utf-8')
                    json_data = json.loads(decoded_line)
                    data_list.append(json_data)
                except ValueError as e:
                    print(f"Error en la linea: {line}")
                    continue
        data_df = pd.DataFrame(data_list)
        data_df.to_csv(file_path1, index = False)
        return print("Archivo guardado satisfactoriamente en :", {file_path1})
    except FileNotFoundError:
        print("El archivo no se encuentra")
        return None
    except Exception as e:
        print(f"Ocurrio un error: {str(e)}")
        return None     

In [10]:
procesar_archivo_json(r'C:\Users\123la\Documents\GitHub\ssss\sesese\data\steam_games.json.gz', r'C:\Users\123la\Documents\GitHub\ssss\sesese\data\games.csv')
procesar_archivo_json(r'C:\Users\123la\Documents\GitHub\ssss\sesese\data\steam_games.json.gz', r'C:\Users\123la\Documents\GitHub\ssss\sesese\data\items.csv')
procesar_archivo_json(r'C:\Users\123la\Documents\GitHub\ssss\sesese\data\steam_games.json.gz', r'C:\Users\123la\Documents\GitHub\ssss\sesese\data\reviews.csv')

Archivo guardado satisfactoriamente en : {'C:\\Users\\123la\\Documents\\GitHub\\ssss\\sesese\\data\\games.csv'}
Archivo guardado satisfactoriamente en : {'C:\\Users\\123la\\Documents\\GitHub\\ssss\\sesese\\data\\items.csv'}
Archivo guardado satisfactoriamente en : {'C:\\Users\\123la\\Documents\\GitHub\\ssss\\sesese\\data\\reviews.csv'}


# Transformación de los datos


### 'games.csv'.


In [2]:
df = pd.read_csv(r'C:\Users\123la\Documents\GitHub\ssss\sesese\data\games.csv', encoding = "utf-8")
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
cantidad_vacios_por_columna = df.isnull().sum()
cantidad_total_vacios = df.isnull().sum().sum()

print("Cantidad de valores vacíos por columna:")
print(cantidad_vacios_por_columna)
print("\nCantidad total de valores vacíos en el DataFrame:", cantidad_total_vacios)


Cantidad de valores vacíos por columna:
publisher       96381
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Cantidad total de valores vacíos en el DataFrame: 1169016


In [4]:
df = df.dropna()
cantidad_vacios_por_columna = df.isnull().sum()
cantidad_total_vacios = df.isnull().sum().sum()

print("Cantidad de valores vacíos por columna:")
print(cantidad_vacios_por_columna)
print("\nCantidad total de valores vacíos en el DataFrame:", cantidad_total_vacios)


Cantidad de valores vacíos por columna:
publisher       0
genres          0
app_name        0
title           0
url             0
release_date    0
tags            0
reviews_url     0
specs           0
price           0
early_access    0
id              0
developer       0
dtype: int64

Cantidad total de valores vacíos en el DataFrame: 0


##### Que columnas conservar


Teniendo en cuenta que se nos pide : 

* def *developer*( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.
Mantener : "tags", "developer"

* def *userdata*( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
Mantener : "price"

* def *UserForGenre*( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
Mantener : "genre", "release_date"

* def *best_developer_year*( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
Mantener : "developer"

* def *developer_reviews_analysis*( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.
Mantener : "developer".

Las columnas a mantener: "developer", "price", "genre", "release_date", "tags", "id", "title". A eliminar: "app_name", "publisher", "url", "early_acces", "specs"

In [35]:
df.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [44]:
df.head()

,genres,title,release_date,tags,price,id,developer
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140.0,Kotoshiro
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",Free To Play,643980.0,Secret Level SRL
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",Free to Play,670290.0,Poolians.com
88313,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,"['Action', 'Adventure', 'Casual']",0.99,767400.0,彼岸领域
88315,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-01-04,"['Action', 'Adventure', 'Simulation', 'FPS', '...",3.99,772540.0,Trickjump Games Ltd


##### Examino la columna "genres"

In [37]:
# Ver el tipo de datos de la columna que contiene genres
print("\nTipo de datos de la columna:")
print(df['genres'].dtype)

# Ver el número de elementos en la columna
print("\nNúmero de elementos en la columna:")
print(len(df['genres']))

# Obtener el tipo de datos único en la columna
print("\nTipos de datos únicos en la columna:")
unique_types = df['genres'].apply(type).unique()
print(unique_types)

# Obtener el tamaño del elemento en cada fila
print("\nTamaño del elemento en cada fila:")
print(df['genres'].apply(lambda x: len(x) if isinstance(x, (dict, list, str)) else None))

# Obtener las claves únicas en todos los genres de la columna
print("\nClaves únicas en los genres:")
unique_keys = set()
for item in df['genres']:
    if isinstance(item, dict):
        unique_keys.update(item.keys())
print(unique_keys)


Tipo de datos de la columna:
object

Número de elementos en la columna:
22511

Tipos de datos únicos en la columna:
[<class 'str'>]

Tamaño del elemento en cada fila:
88310     55
88311     44
88312     59
88313     33
88315     37
          ..
120439    42
120440    45
120441    31
120442    33
120443    19
Name: genres, Length: 22511, dtype: int64

Claves únicas en los genres:
set()


##### Examino la columna "tags"

In [5]:
# Ver el tipo de datos de la columna que contiene tags
print("\nTipo de datos de la columna:")
print(df['tags'].dtype)

# Ver el número de elementos en la columna
print("\nNúmero de elementos en la columna:")
print(len(df['tags']))

# Obtener el tipo de datos único en la columna
print("\nTipos de datos únicos en la columna:")
unique_types = df['tags'].apply(type).unique()
print(unique_types)

# Obtener el tamaño del elemento en cada fila
print("\nTamaño del elemento en cada fila:")
print(df['tags'].apply(lambda x: len(x) if isinstance(x, (dict, list, str)) else None))

# Obtener las claves únicas en todos los tags de la columna
print("\nClaves únicas en los tags:")
unique_keys = set()
for item in df['tags']:
    if isinstance(item, dict):
        unique_keys.update(item.keys())
print(unique_keys)


Tipo de datos de la columna:
object

Número de elementos en la columna:
22511

Tipos de datos únicos en la columna:
[<class 'str'>]

Tamaño del elemento en cada fila:
88310      55
88311     276
88312      74
88313      33
88315     105
         ... 
120439     53
120440     45
120441     31
120442     33
120443     72
Name: tags, Length: 22511, dtype: int64

Claves únicas en los tags:
set()


### 'items.csv